In [3]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [4]:
patients_df = pd.read_parquet("data/PATIENTS.parquet")
patients_df

row_id  subject_id gender        dob        dod   dod_hosp dod_ssn  \
0         234         249      F 2075-03-13        NaT        NaT     NaT   
1         235         250      F 2164-12-27 2188-11-22 2188-11-22     NaT   
2         236         251      M 2090-03-15        NaT        NaT     NaT   
3         237         252      M 2078-03-06        NaT        NaT     NaT   
4         238         253      F 2089-11-26        NaT        NaT     NaT   
...       ...         ...    ...        ...        ...        ...     ...   
46515   31840       44089      M 2026-05-25        NaT        NaT     NaT   
46516   31841       44115      F 2124-07-27        NaT        NaT     NaT   
46517   31842       44123      F 2049-11-26 2135-01-12 2135-01-12     NaT   
46518   31843       44126      F 2076-07-25        NaT        NaT     NaT   
46519   31844       44128      M 2098-07-25        NaT        NaT     NaT   

       expire_flag  
0                0  
1                1  
2                0  
3                0  
4                0  
...            ...  
46515            0  
46516            0  
46517            1  
46518            0  
46519            0  

[46520 rows x 8 columns]

`PATIENTS.parquet` defines all `subject_id`, which is present in the majority of the tables in the dataset.

In [5]:
print(f"There are a total of {patients_df.subject_id.nunique()} unique patients admitted into the ICU")

There are a total of 46520 unique patients admitted into the ICU


`subject_id` is a unique identifier which specifies an individual patient.

`gender` is the sex of the patient.

In [6]:
c = patients_df.gender.value_counts()
p = patients_df.gender.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

counts      %
M   26121  56.15
F   20399  43.85

`dob` is the date of birth of the given patient.

`dod` is the date of death for the given patient (if applicable).

`dod_hosp` is the date of death as recorded in the hospital database.

`dod_ssn` is the date of death from the social security database.

`expire_flag` is a binary flag which indicates whether the patient died, i.e whether `dod` is null or not. These deaths include both deaths within the hospital (`dod_hosp`) and deaths identified by matching the patient to the social security database (`dod_ssn`).

In [8]:
c = patients_df.expire_flag.value_counts()
p = patients_df.expire_flag.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

counts      %
0   30761  66.12
1   15759  33.88